In [1]:
import re
import pandas as pd
from bs4 import BeautifulSoup

### Konwertowanie pliku txt z <b>timestampami na </b> DataFrame

In [2]:
with open('Avengers_Age_of_Ultron(1).txt', 'r') as file:
    content = file.read()

soup = BeautifulSoup(content, 'html.parser')
content_notags = soup.get_text()
dialogue_lines = content_notags.split('\n\n')

starts = []
ends = []
lines = []

for line in dialogue_lines:
    stripped_line = line.split('\n')[1:]
    timestamp = stripped_line[0]
    start = timestamp.split(' --> ')[0]
    end = timestamp.split(' --> ')[1]

    # sometimes 2 people talk at the same time
    sentences = stripped_line[1:]
    for sentence in sentences:
        lines.append(sentence)
        starts.append(start)
        ends.append(end)
    

movie_captions = {
    'line': lines,
    'start': starts,
    'end': ends
}

df = pd.DataFrame(movie_captions)


In [3]:
print(df.head())
df.to_csv('with_timestamps.csv')

                                        line         start           end
0       Report to your stations immediately.  00:00:42,800  00:00:44,802
1  This is not a drill. We are under attack!  00:00:44,960  00:00:47,721
2                       We are under attack!  00:00:49,120  00:00:51,088
3                                    - Shit!  00:01:52,200  00:01:54,487
4                                - Language!  00:01:52,200  00:01:54,487


### Konwertowanie pliku txt z <b>informacją, kto mówi</b> na DataFrame

In [4]:
with open('Avengers_Age_of_Ultron.txt', 'r') as file:
    content = file.read()

speakers = []
lines = []

def get_text(line: str):
    brackets_pattern = r'\[([^]]+)\]'
    text = re.sub(brackets_pattern, '', line)
    return text.strip()


for line in content.split('\n\n'):
    if ':' in line:
        speaker = line.split(':')[0]

        whole_text = get_text(line.split(':')[1])

        whole_text = whole_text.replace('...', ' ').\
                                        replace('?', '.').\
                                             replace('!', '.').\
                                                replace('Dr.', 'Dr ')

        text_splitted = whole_text.split('.')[:-1]
        for sentence in text_splitted:
            if line != '':
                lines.append(sentence)
            speakers.append(speaker)
    else:
        continue


movie_captions = {
    'speaker': speakers,
    'line': lines
}

df = pd.DataFrame(movie_captions)

In [5]:
df.to_csv('who_said_splitted.csv')